In [8]:
import numpy as np
import torch
from torch import nn
from utils import print_losses, load_dataset, load_model, create_model, get_loaders, DATA_DIR, DATASETS

In [9]:
def train_step(batch_x: torch.Tensor, 
               batch_y: torch.Tensor, 
               model: nn.Module, 
               optimizer: torch.optim.Optimizer, 
               criterion
              ):
    optimizer.zero_grad()
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()
    return loss.item()
    
def test_step(batch_x: torch.Tensor, 
              batch_y: torch.Tensor, 
              model: nn.Module, 
              criterion
             ):
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)
    return loss.item()

def train_model(x_train: np.array, 
                y_train: np.array, 
                x_test: np.array, 
                y_test: np.array, 
                model: nn.Module,
                dataset_name,
                epochs: int = 500,
                batch_size: int = 100,
                lr: float = 1e-2,
                early_stopping_rounds: int = 5,
                verbose: bool = True
               ):
    train_loader, test_loader = get_loaders(x_train, y_train, x_test, y_test, batch_size)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_test_loss = 1000000.0
    best_epoch = 0
    
    train_losses = [test_step(batch_x, batch_y, model, criterion) 
                    for batch_x, batch_y in train_loader]
    test_losses = [test_step(batch_x, batch_y, model, criterion) 
                   for batch_x, batch_y in test_loader]
    if verbose:
        print_losses(0, train_losses, test_losses)

    for epoch in range(1, epochs + 1):
        train_losses = [train_step(batch_x, batch_y, model, optimizer, criterion) 
                        for batch_x, batch_y in train_loader]
        test_losses = [test_step(batch_x, batch_y, model, criterion) 
                       for batch_x, batch_y in test_loader]
        if verbose:
            print_losses(epoch, train_losses, test_losses)
        mean_test_loss = np.mean(test_losses)
        if mean_test_loss < best_test_loss:
            torch.save(model.state_dict(), f'best_model_weights-{dataset_name}.pth')
            best_test_loss = mean_test_loss
            best_epoch = epoch
            
        if epoch - best_epoch >= early_stopping_rounds:
            print(f"Early Stopping. Best test loss: {best_test_loss} in epoch {best_epoch}")
            break
        
        

In [ ]:
for dataset_name in DATASETS:
    print("=" * 88)
    x_train, y_train, x_test, y_test, _, _ = load_dataset(dataset_name)
    model = create_model(x_train)
    train_model(x_train, y_train, x_test, y_test, model, dataset_name)

dataset: boston_housing, rows: 506, columns: {x.shape[1]}, range of x: {[x.min(), x.max()]}, range of y: {[y.min(), y.max()]}
Early Stopping. Best test loss: 0.10547972470521927 in epoch 51
dataset: concrete, rows: 1030, columns: {x.shape[1]}, range of x: {[x.min(), x.max()]}, range of y: {[y.min(), y.max()]}
Early Stopping. Best test loss: 0.10901600052602589 in epoch 28
dataset: energy_heating_load, rows: 768, columns: {x.shape[1]}, range of x: {[x.min(), x.max()]}, range of y: {[y.min(), y.max()]}
Early Stopping. Best test loss: 0.04389205574989319 in epoch 89
dataset: kin8nm, rows: 8192, columns: {x.shape[1]}, range of x: {[x.min(), x.max()]}, range of y: {[y.min(), y.max()]}
Early Stopping. Best test loss: 0.10329064975182216 in epoch 108
dataset: naval_compressor_decay, rows: 11934, columns: {x.shape[1]}, range of x: {[x.min(), x.max()]}, range of y: {[y.min(), y.max()]}
Early Stopping. Best test loss: 0.49099695434172946 in epoch 30
dataset: power, rows: 9568, columns: {x.shape[